In [1]:
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import time
import re

API_KEY = '3HutAvbl1TRuIXG0mjgNjTGPuLy_ZhoON7UEYzJqG6o2D6gPkIVZr1RT4x3TBpR3iXtJ9lBnis50H7MXRp0uSisapP-8Tup98qEipjCL6AjzLv2jHM6UhMXZN8IBYnYx'
yelp_api = YelpAPI(API_KEY)

In [2]:
# STEP 1: create list of neighborhoods for each midwest city
cities = ['Chicago, Illinois',
 'Columbus, Ohio', 
 'Indianapolis, Indiana',
 'Detroit, Michigan',
 'Milwaukee, Wisconsin',
 'Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota']    # changed St. Paul to Saint Paul

# scraping wikipedia page
# https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
page = ''
while page == '':
    try:
        page = requests.get('https://en.wikipedia.org/wiki/Lists_of_neighborhoods_by_city#United_States')
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue
        
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
items = content.find_all('li')

In [3]:
# STEP 1.1: get directories (what comes after wikipedia base url) for all cities
# creating dict where keys are cities and values are directories
city_directories = {}

# iterate through list of cities on wikipedia page - if they match our cities, get its directory
for item in items:
    wiki_city_cleaned = item.text.strip('*').strip()
    if wiki_city_cleaned == 'Neighborhoods of Chicago':   # hard-coded
        city_directories[cities[0]] = item.a['href']
    for city in cities:
        if wiki_city_cleaned == city.split(',')[0] or wiki_city_cleaned == city: 
            city_directories[city] = item.a['href']

city_directories['Kansas City, Missouri'] = '/wiki/Neighborhoods_of_Kansas_City,_Missouri'   # hard-coded
city_directories

{'Chicago, Illinois': '/wiki/Neighborhoods_of_Chicago',
 'Columbus, Ohio': '/wiki/Neighborhoods_in_Columbus,_Ohio',
 'Detroit, Michigan': '/wiki/Neighborhoods_in_Detroit',
 'Indianapolis, Indiana': '/wiki/Neighborhoods_of_Indianapolis',
 'Milwaukee, Wisconsin': '/wiki/Neighborhoods_of_Milwaukee',
 'Minneapolis, Minnesota': '/wiki/Neighborhoods_of_Minneapolis',
 'Saint Paul, Minnesota': '/wiki/Neighborhoods_of_Saint_Paul',
 'St. Louis, Missouri': '/wiki/Neighborhoods_of_St._Louis',
 'Kansas City, Missouri': '/wiki/Neighborhoods_of_Kansas_City,_Missouri'}

In [4]:
# STEP 1.2: creating functions for scraping wikipedia page & cleaning
base_url = 'https://en.wikipedia.org'

# uses pandas read_html to scrape neighborhoods from table on each wiki page
def read_wiki(city):
    directory = city_directories.get(city)
    wiki = pd.read_html(base_url + directory)
    return wiki

# uses bs4 to scrape neighborhoods based on html tags
def soup(city):
    directory = city_directories.get(city)
    page = requests.get(base_url + directory)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return content

# cleans where neighborhoods have multiple names with '/'
# NOTE: still in progress

# def cleaned(lst):
#     new_lst = []
#     for neighborhood in lst:
#         if re.search(r"^[\w\s]+\s?[(\/\[]", neighborhood):
#             if "/" in neighborhood:
#                 towns = neighborhood.split("/")
#                 stripped_towns = [i.strip() for i in towns]
#                 new_lst.extend(stripped_towns)
#             elif "[" in neighborhood:
#                 matches = re.findall(r"^([\w\s]+)\s?[(\/\[]", neighborhood)
#                 stripped_matches = [i.strip() for i in matches]
#                 if matches:
#                     new_lst.extend(stripped_matches)
#             else:
#                 new_neighborhood = neighborhood.replace("(", "").replace(")", "").strip()
#                 new_lst.append(new_neighborhood)
#         else:
#             new_lst.append(neighborhood.strip())
#     return new_lst

def cleaned(lst):
    for neighborhood in lst:
        neighborhood = str(neighborhood)
        if '/' in neighborhood or ' / ' in neighborhood:
            lst.remove(neighborhood)
            aliases = neighborhood.split('/')
            for item in aliases:
                lst.append(item.strip())
        if '[' in neighborhood and neighborhood in lst:
            lst.remove(neighborhood)
            split = neighborhood.split('[')
            lst.append(split[0].strip())
    return lst

In [5]:
# STEP 1.3: get neighborhoods for chicago, indianapolis, detroit, st. louis
# creating dict where key is city and value is list of neighborhoods
neighborhoods = {}

# scraping chicago neighborhoods
wiki_chicago = read_wiki('Chicago, Illinois')
chicago_neighborhoods = wiki_chicago[0]
chicago_neighborhoods = list(chicago_neighborhoods['Neighborhood'])
neighborhoods['Chicago, Illinois'] = cleaned(chicago_neighborhoods)

# scraping indianapolis neighborhoods
wiki_ind = read_wiki('Indianapolis, Indiana')
ind_neighborhoods = wiki_ind[2]
indianapolis_neighborhoods = list(ind_neighborhoods[ind_neighborhoods.columns[0]])
neighborhoods['Indianapolis, Indiana'] = cleaned(indianapolis_neighborhoods)

# scraping st. louis neighborhoods
wiki_stl = read_wiki('St. Louis, Missouri')
stl_neighborhoods = wiki_stl[1]
stl_neighborhoods = list(stl_neighborhoods['Neighborhood'])
neighborhoods['St. Louis, Missouri'] = cleaned(stl_neighborhoods)

# scraping detroit neighborhoods
wiki_detroit =  read_wiki('Detroit, Michigan')
detroit_neighborhoods = []
for page in wiki_detroit:
    if 'Name' in page.columns:
        detroit_neighborhoods.extend(list(page['Name']))
        
# cleaning detroit neighborhoods
detroit_neighborhoods = cleaned(detroit_neighborhoods)
for n in detroit_neighborhoods:
    if '/' in n:
        detroit_neighborhoods.remove(n)
        names = n.split('/')
        for name in names:
            detroit_neighborhoods.append(name)
neighborhoods['Detroit, Michigan'] = detroit_neighborhoods

In [21]:
# STEP 1.4: get neighborhoods for columbus, milwaukee, kansas city, minneapolis, saint paul 
# NOTE: need fix repetition of code later for efficiency

# scraping columbus neighborhoods
columbus_directory = city_directories.get('Columbus, Ohio')
page = requests.get(base_url + columbus_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
columbus_neighborhoods = [item.text.replace('[edit]', '') for item in content.find_all(['h2', 'h3', 'h4'])][2:-3]
neighborhoods['Columbus, Ohio'] = cleaned(columbus_neighborhoods)

# scraping milwaukee neighborhoods
mlwk_directory = city_directories.get('Milwaukee, Wisconsin')
page = requests.get(base_url + mlwk_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
mlwk_neighborhoods = [item.text.replace('[edit]', '').strip('"') for item in content.find_all('h3')]
neighborhoods['Milwaukee, Wisconsin'] = cleaned(mlwk_neighborhoods)

# scraping kansas neighborhoods
kansas_directory = city_directories.get('Kansas City, Missouri')
page = requests.get(base_url + kansas_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
li_list = [item.text for item in content.find_all('li')]
kansas_neighborhoods = li_list[li_list.index('CBD-Downtown'):li_list.index('Wornall Homestead') + 1]
neighborhoods['Kansas City, Missouri'] = cleaned(kansas_neighborhoods)

# scraping minneapolis neightborhoods
minneapolis_directory = city_directories.get('Minneapolis, Minnesota')
page = requests.get(base_url + minneapolis_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3 = [item.text.replace('[edit]', '').strip('"') for item in content.find_all(['h3'])]
li = [item.text for item in content.find_all(['li']) if item.text[0].isalpha()]
li = li[0:li.index('University') + 1]
b = [item.text for item in content.find_all('b') if item.text[0].isalpha() and item.text not in li]
minneapolis_neighborhoods = h3 + li + b
neighborhoods['Minneapolis, Minnesota'] = cleaned(minneapolis_neighborhoods)

# scraping saint paul neighborhoods
stpaul_directory = city_directories.get('Saint Paul, Minnesota')
page = requests.get(base_url + stpaul_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3_list = [item.text.strip().split('- ')[1].replace('[edit]', '') for item in content.find_all('h3')]
stpaul_neighborhoods = []
for h3 in h3_list:
    names = h3.split(',')
    for name in names:
        stpaul_neighborhoods.append(name.strip())
neighborhoods['Saint Paul, Minnesota'] = cleaned(stpaul_neighborhoods)

In [37]:
# SETP 2.1: get black owned restaurant id list for Detroit - used to create a column in csv file with Yelp API
black_owned_dtw = []

url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': f'Bearer {API_KEY}', 'user-agent': 'demo'}

for n in neighborhoods['Detroit, Michigan']:
    params = {
        "category": "restaurants",
        "location": f'{n}, Detroit',
        "attributes": "is_black_owned"
    }
    try:
        response = requests.get(url, params=params, headers=headers).json()['businesses']
        for business in response:
            if business['id'] not in black_owned_dtw:
                black_owned_dtw.append(business['id'])
            else:
                continue
    except KeyError:
        continue
        
len(black_owned_dtw)

['quuV34VlUSjFaFWzFDWavw', 'ngur8BN72eO8vh17hW80dg', 'fJ-g62WpfeNbR4qv7qAawA', 'Uwo_Nx_2WDvGF9SwVMlEAg', 'G28_p5q7RnAbeJnIc-XQww', 'VOjmQVpCLjRlOkGtLDLFYQ', 'xuwkaNioH1nttoffOAvYZg', 'ADi8SOow2XPid6NfR98lGQ', 'DGKGnFBhBbT2ZG6Tw4MeSQ', 'XgMQh1K-zbBL0NhPXFi9WA', 'o2yrWVcAqSInuLua4UkK4A', 'Y-b2ciCZx3B66e3jzwM58w', 'fvq6GctVWHECsH3Qr7Cy0w', 'UB69uSqkwjTKMuGbWEqs5w', 'KGflsQO1qzh5V5QfwaDekw', '4in-S0PWC_ESuyWRCySiAQ', 'RMNdJkdMe7jGMQC0ckBlrg', '15OGyKpAqgAJjlXIQUdpqQ', 'gpOKnMXXiRemJf8x0-oHqg', '2ylCgYyEDEVzI0Hve5uggQ', 'HYrqw4xlLCNDptBHGrTIbQ', 'NcM6ASYzFMDN366FsuBTVw', '4p1bXdMdETmyU5qhzFxNxw', 'Xm2A4Xyd-dw4x86qZCV3hw', '8O02CGlIFvl8hm1RCabqhg', 'W8sLqrbS-ZxIuEuQbjAIrw', '8ctLeteWujq_EfH2lRSTAw', '7ArIKwojlm_Yca29HnGDfQ', 'YqmsCvd3w0RxEiAcYGj64w', 'x7bzkyPv0pf8mmp4BaexZQ', 'UdO_uBm3sIM5DwaE8XIpbg', '3Nbs9f7DxP3siSu1pjLj1w', 'Qn3BDpni0V2vtp9P0-85BQ', 'WFtnK-JJnoVHtbMAQW7tNg', 'YtWT6_MKURJKEpNKbO-nzg', 'gL6l1EqphkyLqER5lgmReg', '65pRcC4awElAnzH4io9Chw', 'M2XMDFFwed7IFCJ9Ey6Mhw', 'ngu3My-XGz

In [40]:
# STEP 2.3: use yelp API to get restaurant data for each neighborhood and write results into csv file

# way to eliminate duplicates
previous_restaurant_ids = []

with open('detroit_restaurants.csv', "w", newline="") as fileout:
    header = fileout.write("id,name,review_count,rating,latitude,longitude,price,zip_code,is_black_owned\n")
    writer = csv.writer(fileout)
    count = 0
    for n in neighborhoods['Detroit, Michigan']:
        offset = 0
        try:
            # obtaining up to 500 restaurants per neighborhood
            while offset <= 500:
                result = yelp_api.search_query(categories='restaurants', location=n + ', Detroit', limit=50, offset=offset)['businesses']
                # parsing through json
                for business in result:
                    restaurant_id = business['id']
                    name = business['name']
                    review_count = business['review_count']
                    rating = business['rating']
                    latitude = str(round(business['coordinates']['latitude'], 5))
                    longitude = str(round(business['coordinates']['longitude'], 5))
                    zip_code = business['location']['zip_code']
                    try:
                        price = business['price']
                    except KeyError:
                        price = 'NA'
                    # writing to csv file with conditions - whether black owned or duplicate
                    if restaurant_id not in previous_restaurant_ids and restaurant_id in black_owned_dtw:
                        is_black_owned = 1
                        row = restaurant_id,name,review_count,rating,latitude,longitude,price,zip_code,is_black_owned
                        writer.writerow(row)
                        count += 1
                        print(name + ' - '+ str(count))
                        previous_restaurant_ids.append(restaurant_id)
                    elif restaurant_id not in previous_restaurant_ids and restaurant_id not in black_owned_dtw:
                        is_black_owned = 0
                        row = restaurant_id,name,review_count,rating,latitude,longitude,price,zip_code,is_black_owned
                        writer.writerow(row)
                        count += 1
                        print(name + ' - '+ str(count))
                        previous_restaurant_ids.append(restaurant_id)
                    else:
                        print('Skipping duplicates...')
                    offset += 50
        # way to handle errors if neighborhood does not give search results with Yelp API
        except YelpAPI.YelpAPIError as e:
            print('Error')
            continue
    print('Done')

Jacoby's German Biergarten - 1
Sweetwater Tavern - 2
Niki's Pizza - 3
Detroiter Bar - 4
Floods Bar & Grille - 5
The Elephant Room - 6
Pegasus Taverna - Greektown - 7
Red Smoke Barbecue - 8
Loco's Tex-Mex Grille - 9
The Clique Restaurant - 10
Grand Trunk Pub - 11
Basement Burger Bar - 12
Central Kitchen + Bar - 13
Nathan's Deli - 14
Prism - 15
Firebird Tavern - 16
Townhouse - 17
Wahlburgers - 18
Besa - 19
Golden Fleece Restaurant - 20
Fishbone's - 21
Highlands - 22
Checker Bar - 23
The Greek - 24
Ham Shop Cafe - 25
Eat-N-Out Pizza & Gelato - 26
Orchid Thai Restaurant - 27
Cadillac Square Diner - 28
IHOP - 29
Lunchtime Global - 30
Bangkok Crossing - 31
Harbor House - 32
The Old Shillelagh - 33
Applebee's Bar + Grill - 34
Mr. Kabob Xpress - 35
Minnie's Detroit - 36
Bistro 555 - 37
Volt - 38
Gourmet Deli - 39
Level Two Bar & Rooftop - 40
The Baltimore - 41
Shredderz Food Truck - 42
Delmar - 43
Delectabowl Food Truck & Catering - 44
Buffalo Wild Wings - 45
Hearth 71 - 46
Woodward Coney Rest

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping du

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping du

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping du

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping du

...

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Ivy Kitchen + Cocktails - 526
Sindbads Restaurant & Marina - 527
Legends Grill - 528
Southern Fires Bistro & Lounge - 529
Jet's Pizza - 530
Pekin Pavilion - 531
Metropolitan Bar And Kitchen - 532
Tai Hing - 533
Detroit Grill & Chill - 534
Gabriel Hall - 535
McDonald's - 536
Wendy's - 537
Grandy's Coney Island - 538
Krispy Krunchy Chicken - 539
Peking Express - 540
Little Caesars - 541
Happy's Pizza - 542
Planted Detroit - 543
Church's Chicken - 544
Little Caesars Pizza - 545
Wendy's - 546
Palace Pizza - 547
Hungry House - 548
21 Flavorz Express - 549
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skippi

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Chops Grill - 668
Skipping duplicates...
Skipping duplicates...
Cajun Boiling Crab - 669
Good Cookies - 670
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Cheng's Garden Restaurant - 671
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Domino's Pizza - 672
Skipping duplicates...
Skipping duplicates...
Street Cuisine - 673
Skipping duplicates...
Church's Chicken - 674
Detroit Taco Bar - 675
Skipping duplicates...
Burger King - 676
Skipping duplicates...
Skipping duplicates...
Asian Corned Beef - 677
Skipping duplicates...
Skipping duplicates...
Skipping duplicat

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
La Noria Bistro - 844
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping dup

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Papa's Pizza & Bbq - 875
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Wing Snob - 876
Buddy's Pizza - Grosse Pointe - 877
White Castle - 878
Alexander & Polen Meat Market - 879
Big Burgzs - 880
Mr C's Deli of Grosse Pointe Farms - 881
Einstein Bros. Bagels - 882
National Coney Island - 883
Skipping duplicates...
Skipping duplicates...
Rob's Deli - 884
Jet's Pizza - 885
Detroit Wing - 886
Popeyes Louisiana Kitchen - 887
McDonald's - 888
Skipping duplicates...
Burger Pointe and Wings - 889
Skipping duplicates...
Sweetwater Express - 890
Wendy's - 891
Burger King - 892
Papa Johns Pizza - 893
Chipotle Mexican Grill - 894
Firewa

Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Skipping duplicates...
Conant Cafe' & Grill - 943
Alqamar - 944
Skipping duplicates...
Jet's Pizza - 945
Boostan Cafe Detroit - 946
Skipping duplicates...
Joe Ann's Bar B-Q - 947
Nevada A Coney I